In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
#from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df=pd.read_csv("C:/Users/surendar.reddy/Documents/data Science projet_Data traine/DataTrained_Project_Week-1/Advertising.csv")
pd.set_option('display.max_rows', None)
df

In [ ]:
df.shape

In [ ]:
df.info()
#No Null Values
#Unnamed looks a default columns
#No need Imputation or Encoding required for the dataset

In [ ]:
df.describe()
#Data looks okay

In [ ]:
df.isnull().sum()

In [ ]:
df.drop(['Unnamed: 0'],inplace=True, axis=1)

In [ ]:
#Check how the data looks, with respect to distribution
plt.figure(figsize=(20,20),facecolor='white')
plotnumber=1
for column in df:
    if plotnumber<=12:
        ax=plt.subplot(3,4,plotnumber)
        sns.distplot(df[column])
        plt.xlabel(column,fontsize=20)
    plotnumber+=1
plt.tight_layout()

In [ ]:
df.skew()
#Only Newpaper data requires correlation correction

In [ ]:
#check for the correlation aspect between the fetures
corr=df.corr()
plt.figure(figsize=(3,3))
sns.heatmap(corr,annot=True)
plt.show()
#Advertisment in TV is more correlated to Sales, then Radio and then Newpaper

In [ ]:
#We do not need to check for multicollinerity as there are 3 means of Adversiment for Total sales
plt.figure(figsize=(20,20),facecolor='white')
plotnumber=1
for column in df:
    if plotnumber<=12:
        ax=plt.subplot(3,4,plotnumber)
        sns.scatterplot(df[column],df['sales'])
        plt.xlabel(column,fontsize=20)
    plotnumber+=1
plt.tight_layout()

In [ ]:
x=df.drop(columns=['sales'],axis=1)
y=df['sales']

In [ ]:
#Let do trasformation of the features o reduce screwness
#Screwness Correction
from sklearn.preprocessing import power_transform
x_new=power_transform(x)
x=pd.DataFrame(x_new,columns=x.columns)
x.skew().sort_values(ascending=False)
#Now the screwness is rectified from Fetures
#These values look more convincing

In [ ]:
plt.figure(figsize=(20,20),facecolor='white')
plotnumber=1
for column in x:
    if plotnumber<=12:
        ax=plt.subplot(3,4,plotnumber)
        sns.distplot(x[column])
        plt.xlabel(column,fontsize=20)
    plotnumber+=1
plt.tight_layout()

In [ ]:
scaler=StandardScaler()
x_scaled=scaler.fit_transform(x)
x_scaled

In [ ]:
#Train the data to the model
x_train, x_test, y_train, y_test=train_test_split(x_scaled,y,test_size=0.25, random_state=100)

In [ ]:
from sklearn.linear_model import LinearRegression
reg=LinearRegression()
reg.fit(x_train,y_train)
y_pred=reg.predict(x_test)
reg_accuracy=metrics.r2_score(y_test,y_pred)
print(reg_accuracy)

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
ada=AdaBoostRegressor()
ada.fit(x_train,y_train)
y_pred=ada.predict(x_test)
ada_accuracy=metrics.r2_score(y_test,y_pred)
print(ada_accuracy)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
RFR=RandomForestRegressor()
RFR.fit(x_train,y_train,sample_weight=100)
y_pred=RFR.predict(x_test)
RFR_accuracy=metrics.r2_score(y_test,y_pred)
print(RFR_accuracy)

In [ ]:
#Lets check if the RF accuracy is not overfitting the model

from sklearn.model_selection import KFold,cross_val_score
k_f=KFold(n_splits=5, shuffle=True)
k_f
x_scaled=pd.DataFrame(x_scaled,columns=x.columns)
from sklearn.metrics import accuracy_score
k=5
acc_score = []

for train_index , test_index in k_f.split(x_scaled):
    x_train , x_test = x_scaled.iloc[train_index,:],x_scaled.iloc[test_index,:]
    y_train , y_test = y[train_index] , y[test_index]
     
    RFR.fit(x_train,y_train)
    pred_values = RFR.predict(x_test)
     
    acc = metrics.r2_score(y_test,pred_values)
    acc_score.append(acc)
     
avg_acc_score = sum(acc_score)/k
 
print('accuracy of each fold - {}'.format(acc_score))
print('Avg accuracy : {}'.format(avg_acc_score))

In [ ]:
#This model looka good, The cross validation scores and actual scores are not very different
#will Hypertune the model if we can improve the score

In [ ]:
from sklearn.model_selection import GridSearchCV

RFR=RandomForestRegressor()
params={'n_estimators':[60,70,90],
        'criterion':['mse','gini'],
        'max_depth':[10,11,12,13,14,15,16,17,18,19,20],
         'min_samples_split':[10,11,15],
         'min_samples_leaf':[4,5]
       }
grd=GridSearchCV(RFR,param_grid=params,)
grd.fit(x_train,y_train)
RFR=grd.best_estimator_
y_pred=RFR.predict(x_test)
RFR_accuracy=metrics.r2_score(y_test,y_pred)
RFR_accuracy

In [ ]:
#Could not improve much on Hypertunning aftermultiple attems also.
with open("RFR", "wb") as f:
    pickle.dump(svc, f)